In [41]:
import requests
import bs4
from bs4 import BeautifulSoup

import pandas as pd
import time
import copy
import csv

In [42]:
def get_url(position):
  '''Function to search a position on the Indeed webpage'''
  template = 'https://www.indeed.com/jobs?q={}&l'
  url = template.format(position)
  
  return url

In [43]:
def response_func(url):
  '''Function returns a get request for html of an indeed webpage'''
  response = requests.get(url, headers={'User-Agent': 'test'})

  return response

In [44]:
def get_cards(response):
  ''' Indeed search results are divided up by cards.
      This function returns the cards get_job_cards for each job title'
  '''
  soup = BeautifulSoup(response.text, 'html.parser')
  cards = soup.find_all('td', {'class': 'resultContent'})

  return cards

In [45]:
def get_job_title(card):
  '''Function returns the job title'''
  job_title = card.find_all('span')[1].get_text()
  
  return job_title

In [46]:
def get_job_url(card):
  '''Function returns the job posting url'''
  try:
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')
    return job_url
  except AttributeError:
    return 'https://www.indeed.com/404'

In [47]:
def get_company_name(card):
  '''Function returns the company name who posted the job'''
  company = card.find('span', 'companyName').text

  return company

In [48]:
def get_location(card):
  '''Function returns the job location'''
  location = card.find('div', 'companyLocation').text

  return location

In [49]:
def get_job_description(job_url):
  response = requests.get(job_url, headers={'User-Agent': 'test'})
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    description = soup.find('div', 'jobsearch-jobDescriptionText').text.strip()
    return description
  except AttributeError:
    return "No job description"

In [50]:
def get_job_records(position):
  '''
  Function returns an array of the following
  * Company Name
  * Job Title
  * Location
  * Descriptions
  * Job Url
  '''
  # Retrieves the data on the first page of the search results

  records = []
  url = get_url(position)
  response = response_func(url)
  while True:
    cards = get_cards(response)
    for card in cards:
      job_title = get_job_title(card)
      company_name = get_company_name(card)
      job_location = get_location(card)
      job_url = get_job_url(card)
      job_description = get_job_description(job_url)

      records.append([job_title,
                      company_name,
                      job_location,
                      job_description,
                      job_url])
    # Goes through each page of the search results
    try:
      response = requests.get(url)
      soup = BeautifulSoup(response.text, 'html.parser')
      url = 'https://www.indeed.com' + soup.find(
                        'a',
                        {'aria=label': 'Next'}
                        ).get('href')
    except AttributeError:
      break
  return records


In [51]:
search_terms = [
    "data%20scientist",
    "machine%20learning%20engineer",
    "data%20engineer",
    "web%20developer",
    "frontend%20developer",
    "backend%20developer",
    "devops",
    "software%20engineer",
]

In [52]:
from random import uniform
for position in search_terms:
  time.sleep(uniform(1.1,2.1))
  try:
    scraped_list = get_job_records(position)
    indeed_jobs = pd.DataFrame(scraped_list,
                              columns=[
                                        'Company',
                                        'JobTitle',
                                        'Location',
                                        'Description',
                                        'JobUrl'])
    saving_jobs = copy.deepcopy(indeed_jobs)
    saving_jobs = saving_jobs.drop_duplicates(['Description']).reset_index(drop=True)
    saving_jobs = saving_jobs[saving_jobs['Description'].notna()]
    if len(saving_jobs)==1:
      continue
    display(saving_jobs)

    position = position.replace('%20', '_')

    saving_jobs.to_csv(f'{position}_jobs.csv', encoding='utf-8', index=False)
    print(f'Successfully saved {position} jobs!")')
  except AttributeError:
    print(f'Failed to save {position} jobs check errlogs')
    continue

,Company,JobTitle,Location,Description,JobUrl
0,Data Scientist,Jones Family Office,Remote,Who We Are: SumerSports is a fast-growing NFL ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Jr. Data Scientist,Talentheed Inc,Remote,Responsibilities: \n\nData mining or extricati...,https://www.indeed.com/company/Talentheed-Inc/...
2,"GSSA, LLC","GSSA, LLC",Hybrid remote,Employment InformationJob Title: Data Scientis...,"https://www.indeed.com/company/GSSA,-LLC/jobs/..."
3,Union Pacific,Union Pacific,"Omaha, NE",UP’s Advanced Analytics team is looking for a ...,https://www.indeed.com/rc/clk?jk=9b9110ff2e117...
4,Data Scientist,S&P Global,"Remote in Bennington, NE 68007+43 locations",Segment: Market Intelligence \n \nThe Role: Da...,https://www.indeed.com/rc/clk?jk=bbd54e0e7ce2b...
5,Data Scientist,Toyota,"Remote in Plano, TX 75023+4 locations",Who we’re looking for:\nThe Data Scientist wil...,https://www.indeed.com/rc/clk?jk=e6f31f3bbe73b...
6,Nebraska Furniture Mart,Nebraska Furniture Mart,"Omaha, NE 68114+1 location","At Nebraska Furniture Mart (NFM), we’ve been h...",https://www.indeed.com/rc/clk?jk=c92daa0424b88...
7,Data Scientist,Constellation West,"Offutt AFB, NE 68113",Constellation West is seeking a Data Scientist...,https://www.indeed.com/rc/clk?jk=685e3a3e4c54e...
8,Madison Square Garden Entertainment,Madison Square Garden Entertainment,"Remote in Tarrytown, NY 10591",Who are we hiring?\n\n As a Basketball Data Sc...,https://www.indeed.com/rc/clk?jk=41d2ea80ddeb8...
9,Data Scientist,GSK,"Rogers, AR 72758+20 locations",Site Name: USA - Arkansas - Bentonville\nPoste...,https://www.indeed.com/rc/clk?jk=32f70c24a0a62...


Successfully saved data_scientist jobs!")


,Company,JobTitle,Location,Description,JobUrl
0,"Manager, Data Warehouse Engineer",Indeed,+4 locationsRemote,No job description,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Data Engineer + Apprentice (Entry-Level),New Apprenticeship,"Jersey City, NJ",Job Description: \nAre you passionate about da...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,Union Pacific,Union Pacific,"Omaha, NE",Union Pacific is embarking on a digital transf...,https://www.indeed.com/rc/clk?jk=d96bdfa099a51...
3,Data Engineer,workcog,+3 locationsRemote,Role: Data Engineer\nLocation: Remote\nDuratio...,https://www.indeed.com/company/WorkCog/jobs/Da...
4,Data Engineer,Facet Wealth,Remote,Facet is a fully remote FinTech company with a...,https://www.indeed.com/rc/clk?jk=37aa4efd27979...
5,Netflix,Netflix,+3 locationsRemote,"Remote, United States\nData Science and Engine...",https://www.indeed.com/rc/clk?jk=1699b30e2ac8c...
6,Tesla,Tesla,+8 locationsRemote,The Data Engineer is responsible for processin...,https://www.indeed.com/rc/clk?jk=c88b779349d52...
7,Data Solutions Engineer,Samsung Electronics,"Austin, TX 78754",Position Summary \nSamsung Austin Semiconducto...,https://www.indeed.com/rc/clk?jk=7387ac9a16c30...
8,Data Engineer (Fully Remote),BOEING,"Remote in Seattle, WA 98164+35 locations","Job Description\nAt Boeing, we innovate and co...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
9,Data Engineer at Rent-to-own Startup,Recruiting From Scratch,"Remote in Omaha, NE 68144+126 locations",Who is Recruiting from Scratch:\n \n\n ...,https://www.indeed.com/rc/clk?jk=4067ff8079ed3...


Successfully saved data_engineer jobs!")


,Company,JobTitle,Location,Description,JobUrl
0,Database Administrator/Web Developer,University of Nebraska Medical Center,"Hybrid remote in Omaha, NE",Department CON-Continuing Education 50001255 B...,https://www.indeed.com/rc/clk?jk=f89350b645d23...
1,Junior Web Developer,Scorpion Internet Marketing,Remote,"As a Junior Web Developer, you are responsible...",https://www.indeed.com/company/Scorpion-Intern...
2,1Starr,1Starr,Remote,1Starr Enterprises looking for Web Designers w...,https://www.indeed.com/company/1Starr/jobs/Web...
3,Jr. Web Developer,"Quality Liquid Feeds, Inc","Dodgeville, WI 53533","Quality Liquid Feeds, Inc. is looking to add a...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4,Web Designer,Herbsmith,Remote,Who are we? What does Herbsmith Inc. stand for...,https://www.indeed.com/company/Herbsmith/jobs/...
5,The Public Interest Network,The Public Interest Network,"Denver, CO 80202 (Union Station area)",Developer\n*Are you a tech-savvy person who ca...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
6,Crunchapps,Crunchapps,"New York, NY 10014 (West Village area)",Join an Innovative Deep Learning team with the...,https://www.indeed.com/rc/clk?jk=9e76e485b48fe...
7,DashForth,DashForth,"Salt Lake City, UT 84104",We will guarantee to beat your current salary ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
8,Web Designer,Elkhorn Computer Sales & Service,"Elkhorn, NE 68022",No job description,https://www.indeed.com/company/Elkhorn-Compute...
9,Entry Level Software Developer,Property Matrix,"Irvine, CA+8 locations",Minimum Qualifications: \n\nCompleted bachelor...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


Successfully saved web_developer jobs!")


,Company,JobTitle,Location,Description,JobUrl
0,Apple,Apple,"Austin, TX+23 locations","Summary\nPosted: Oct 20, 2021\nWeekly Hours: 4...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Entry Level Unix/Linux DEVOPs Admin,Commonwealth of Kentucky,"Frankfort, KY 40601",Entry Level Unix/Linux DevOps Admin\nFrankfort...,https://www.indeed.com/company/Harvey-Nash/job...
2,The Mom Project,The Mom Project,Remote,Our Customer is an independent investment mana...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,DevSecOps Developer,Agensys Corporation,Remote,Responsibilities: \n· Automate the development...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4,DevOps Engineer - 4187460,Accenture,"Charlotte, NC 28244 (Downtown Charlotte area)+...",Accenture Flex offers you the flexibility of l...,https://www.indeed.com/rc/clk?jk=f5771784c0b7f...
5,DevOps Engineer (100% Remote),CyberCoders,"Remote in Boulder, CO 80302+126 locations",DevOps Engineer (100% Remote) \n \nDevOps Eng...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
6,Remote DevOps/SRE Engineer,Ascendo Resources,+7 locationsRemote,This is an exciting opportunity for someone wi...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
7,DevOps Engineer,Volt Workforce Solutions,+4 locationsRemote,Volt is immediately hiring DevOps Engineer/SRE...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
8,Remote Junior DevOps Engineer,Siemens,Remote in United States+24 locations,Position Overview\n Siemens Government Technol...,https://www.indeed.com/rc/clk?jk=6fc3747e192d4...
9,REMOTE- Sr. DevOps (up to 180k),Vaco,"Remote in Cincinnati, OH+6 locations",DevOps Engineer\n Full-time role\n Remote \nAb...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


Successfully saved devops jobs!")


,Company,JobTitle,Location,Description,JobUrl
0,Software Engineer I/II,Microsoft,"Redmond, WA 98052 (Overlake area)+126 locations",Microsoft is on a mission to empower every per...,https://www.indeed.com/rc/clk?jk=713033755e8b4...
1,University of Nebraska Medical Center,University of Nebraska Medical Center,"Remote in Omaha, NE",Department IT Development and Operations 50000...,https://www.indeed.com/rc/clk?jk=fbaea9645a432...
2,Junior Software Engineer,Stellantis,Remote,Job ID: 1047397\nCareer Area: Information Tech...,https://www.indeed.com/rc/clk?jk=01eae2b0386a1...
3,Software Engineer II,University of Nebraska Medical Center,"Remote in Omaha, NE+1 location",Department UNeTech 50012027 Business Unit Busi...,https://www.indeed.com/rc/clk?jk=d19b30f3c5b44...
4,Avant,Avant,Remote,Avant is dedicated to building premier digital...,https://www.indeed.com/company/Avant/jobs/Entr...
5,Sticker Mule,Sticker Mule,Remote,About Sticker Mule\n\n\n Sticker Mule is th...,https://www.indeed.com/rc/clk?jk=1756e7b0298d0...
6,Associate Software Engineer - Virtual Simulation,BOEING,"Berkeley, MO 63134+126 locations",No job description,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
7,NBCUniversal,NBCUniversal,Remote,69366BR\nTechnology & Engineering\nOperations ...,https://www.indeed.com/rc/clk?jk=589e7df855f13...
8,Insurity,Insurity,Remote,Software Development at Insurity\nFrom Softwar...,https://www.indeed.com/rc/clk?jk=676599852a737...
9,ACI Worldwide,ACI Worldwide,"Remote in Omaha, NE 68102+2 locations",Join the Team Making Possibilities Happen \nIf...,https://www.indeed.com/rc/clk?jk=ae9008d2f6729...


Successfully saved software_engineer jobs!")


# New Section